In [7]:
from bs4 import BeautifulSoup

# Method to parse the structure of an html page using package beautifulsoup.
# The code looks for specific tags in the html structure and extracts the content
def getArticleDetailsByUrl(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,"html.parser")
    #soup.prettify()
    
    headline = soup.title.string
    subheadline = soup.head.find("meta",attrs={"name":"description"}).get('content')

    doc_body = ''
    if "The Irish Times" in soup.text:
        for body_p_tag in soup.article.find_all("p", attrs={"class": "no_name"}):
            doc_body += body_p_tag.get_text() + " "

    source = "Other"
    try:
        if "irishtimes" in url:
            source = "IrishTimes"
            body_p_tag = soup.article.find("div", attrs={"class": "last_updated"}).find("p")
    except:
        pass

    first_sentence = doc_body.split(".")[0]

    return [headline, subheadline, first_sentence, doc_body, source]



In [8]:
import requests
#Feedparser is a library to parse RSS/XML feeds, these are files with a specific XML structure
#If you don't have it, install using conda or pip, e.g.,: pip install feedparser
import feedparser
#help(feedparser)

#Parse the XML file to retrieve the URLs for individual news articles.
#Parse each article's HTML page
def scrapRSSFeed(rss_feed):
    count= 0
    article_number = []
    article_title = []
    article_content = []
    d = feedparser.parse(rss_feed)
    #print(d)
    #print(d['entries'], "\n")
        
    for item in d['entries']:
        #Extract an article URL
        article_url = item['link']
        [headline, subheadline, first_sentence, doc_body, source] = getArticleDetailsByUrl(article_url) 
        article_number.append(count)
        count+=1
        article_title.append(headline)
        article_content.append(doc_body)
        
    return article_number,article_title,article_content

In [9]:
#The URL of the XML file
import pandas as pd
url='http://www.irishtimes.com/cmlink/news-1.1319192'
xml_page = requests.get(url).text

#Look at the structure of the XML file
#To have a proper look, open the XML file with a text editor

# print('The XML content: \n',xml_page[:1000])
# print('-'*100)


# Call the method that parses a given XML file
# print('Just print the Headline of articles: \n')
number,title,content = scrapRSSFeed(url)

number = [str(item).zfill(3) for item in number]


df = pd.DataFrame({
    'Article ID':number,
    'Title' : title,
    'Content':content
})

df  = df.set_index('Article ID')
df

,Content,Title
Article ID,,
000,The top prizes in this year’s European Union C...,EU Contest for Young Scientists features starc...
001,A few years ago entrepreneur and multimilliona...,Junior entrepreneurs encouraged to start real ...
002,A report has called for new guidelines for jud...,New personal injury measures needed to reduce ...
003,A major programme of compulsory purchasing of...,Councils should buy vacant houses compulsorily...
004,When Alvaro Cabello came to Ireland in 2012 it...,Unemployment main driver for Spaniards moving ...
005,Failure rates in some treatments using “surgi...,"Surgical mesh failure rates unacceptably high,..."
006,An 83-year-old man who helped to thwart an att...,Man (83) ‘hadn’t much choice’ but to try stop ...
007,Minister for Justice Charlie Flanagan has said...,Minister cautions that report on future of pol...
008,The rate of uptake of the cervical cancer vacc...,Cervical cancer vaccine uptake rises to 65%
